In [ ]:
import cv2 # pip install opencv-python
from HandTrackingModule import HandDetector
from DetectorModule import DetectorThread
import time 
# import paho.mqtt.client as mqtt #pip install paho-mqtt


def on_connect(client, userdata, flags, rc): 
   print("Connected with result code " + str(rc)) 
   client.subscribe("/leds/pi") 
def on_message(client, userdata, msg): 
   print(msg.topic+" "+str( msg.payload))  
# client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION1) 
# client.on_connect = on_connect 
# client.on_message = on_message 
# client.connect('192.168.43.176', 1883, 60) 
# client.loop_start() 
print('Script is running, press Ctrl-C to quit...')


ACTIONS = {"All Leds On":[[1,1,1,1,1],[1,1,1,1,1]],
           "All Leds Off":[[0,0,0,0,0],[0,0,0,0,0]],
           "Led 1 On":[[1,1,1,1,1],[]],
           "Led 2 On":[[],[1,1,1,1,1]],
           "Led 1 Off":[[0,0,0,0,0],[]],
           "Led 2 Off":[[],[0,0,0,0,0]],
           "Fan On":[[],[0,1,1,0,0]],
           "Fan Off":[[],[0,1,0,0,0]],
           "Object On":[[0,1,1,0,0],[]],
           "Object Off":[[0,1,0,0,0],[]]
           }


detector=HandDetector(detectionCon=0.8,maxHands=2,glasses=True)
object_detector = DetectorThread("detector/model.tflite","detector/coco_labels.txt")

video=cv2.VideoCapture(0)
f_type = 0
c_type = 0
start_time = time.time()

action = "Current action: "
color  = (255,255,255) 
action = ""
last_action_sent = ""
run_object_detector = False

while True:
    delay = time.time()
    ret,frame=video.read()
    frame = cv2.resize(frame,(640,480))
    cp_frame = frame.copy()
    if not ret:
        break
    hands,handsType,img=detector.findHands(frame)
    if hands:
        if len(hands)==2:
            lmList1=hands[0]
            lmList2=hands[1]
            fingerUp1=detector.fingersUp(lmList1)
            fingerUp2=detector.fingersUp(lmList2)
            f_type = [fingerUp1,fingerUp2]
        else:
            lmList=hands[0]
            fingerUp=detector.fingersUp(lmList)
            if handsType[0] == "Right":
                f_type = [fingerUp,[]]
            else:
                f_type = [[],fingerUp]
        # print(f_type)
        if f_type == c_type:
            elapsed_time = time.time() - start_time
            if elapsed_time >=1:
                try:
                    action = "Current action: " + next(key for key, value in ACTIONS.items() if value == f_type)
                except StopIteration:
                    continue
        else:
            start_time = time.time()
            c_type = f_type
    cv2.putText(frame,action,(0,50),cv2.FONT_HERSHEY_COMPLEX,1,color,1,cv2.LINE_AA)
    cv2.imshow('frame',frame)
    if run_object_detector:
        object_detector.recognize_objects(cp_frame)
        cp_frame = cv2.resize(cp_frame,(640,480))
        cv2.imshow("object detector",cp_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    if action == last_action_sent:
        continue
    if "all" in action.lower() and "on" in action.lower():
        color = (0,255,255)
        time.sleep(0.02)  # Delay for about 20 milliseconds to debounce. 
        client.publish('/leds/esp8266', 'ALLON') 
    if "all" in action.lower() and "off" in action.lower():
        color = (0,255,255)
        time.sleep(0.02)  # Delay for about 20 milliseconds to debounce. 
        client.publish('/leds/esp8266', 'ALLOFF') 
    if "1" in action.lower() and "on" in action.lower():
        color = (0,0,255)
        time.sleep(0.02)  # Delay for about 20 milliseconds to debounce. 
        client.publish('/leds/esp8266', 'LED1ON') 
    if "1" in action.lower() and "off" in action.lower():
        color = (0,0,255)
        time.sleep(0.02)  # Delay for about 20 milliseconds to debounce. 
        client.publish('/leds/esp8266', 'LED1OFF') 
    if "2" in action.lower() and "on" in action.lower():
        color = (0,255,0)        
        time.sleep(0.02)  # Delay for about 20 milliseconds to debounce. 
        client.publish('/leds/esp8266', 'LED2ON') 
    if "2" in action.lower() and "off" in action.lower():
        color = (0,255,0)
        time.sleep(0.02)  # Delay for about 20 milliseconds to debounce. 
        client.publish('/leds/esp8266', 'LED2OFF')         
    if "fan" in action.lower() and "on" in action.lower():
        color = (255,255,255)
        time.sleep(0.02)  # Delay for about 20 milliseconds to debounce. 
        client.publish('/leds/esp8266', 'FANON') 
    if "fan" in action.lower() and "off" in action.lower():
        color  = (255,255,255)
        time.sleep(0.02)  # Delay for about 20 milliseconds to debounce. 
        client.publish('/leds/esp8266', 'FANOFF') 
    if "object" in action.lower() and "on" in action.lower():
        run_object_detector = True
    if "object" in action.lower() and "off" in action.lower():
        run_object_detector = False
        if cv2.getWindowProperty("object detector", cv2.WND_PROP_VISIBLE) > 0:
            cv2.destroyWindow("object detector")
        
    last_action_sent = action
    action = ""
    
cv2.destroyAllWindows()

video.release()


In [7]:
import cv2
from DetectorModule import DetectorThread

object_detector = DetectorThread("detector/model.tflite","detector/coco_labels.txt")

cp_frame = cv2.imread("test_images/laptop.jpg")
cp_frame = cv2.resize(cp_frame,(640,480))
object_detector.recognize_objects(cp_frame)
cv2.imshow("object detector",cp_frame)

cv2.waitKey(0)
cv2.destroyAllWindows()

2.9114773273468018
2.7545933723449707
